In [ ]:
from glob import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
from tqdm import tqdm
from simulation import SimulationLeadInteraction
from load_interactions import load_interaction
from stats import KDE
%matplotlib inline

In [ ]:
files = glob(os.path.join("data", "8_interactions", "*.hdf5"))

In [ ]:
x = load_interaction(files[0])

In [ ]:
def get_pars(vel_acc):
    if np.max(np.abs(np.diff(vel_acc['vel_x']))) > 1.5:
        return np.zeros((0, 4))
    
    data = vel_acc.copy()
    i = data.index[scipy.signal.find_peaks(-data['vel_x'], prominence=1)[0]]
    data['endspeed'] = np.nan
    data['endtime'] = np.nan
    data.loc[i, 'endspeed'] = data.loc[i, 'vel_x']
    data.loc[i, 'endtime'] = i
    data = data.fillna(method='backfill')
    data = data.dropna()

    data['duration'] = (data['endtime'] - data.index) / 10
    data['reduction'] = data['vel_x'] - data['endspeed']
    data['dmean'] = data['reduction'] / data['duration']
    data = data.drop(i)
    return data[['vel_x', 'acc_x', 'reduction', 'dmean']].values[::10]

In [ ]:
parameters = [get_pars(load_interaction(file)['vehicle_front']) for file in files[:100]]

In [ ]:
kde = KDE(np.concatenate(parameters), scaling=True)
kde.clustering()
kde.compute_bandwidth()

In [ ]:
pars = np.concatenate(parameters)

In [ ]:
kde = KDE(pars, scaling=True)
kde.clustering()
kde.data.shape

In [ ]:
kde.compute_bandwidth()

In [ ]:
kde.bandwidth